In [175]:
import pandas as pd
import datetime
import math

#import os
excel_file = 'SIP.xlsx'
#excel_file = '/Users/jbanerjee/Desktop/SIP2.xlsx'

#read excel file
sip = pd.read_excel(excel_file, 'Main')

#change it to list
table = sip.values.tolist()


#change to dataframe
df = pd.DataFrame(table[1:], columns=table[0])

#select only those 2 columns that is required to do the job
df = df.iloc[:, [0,7]]

date_time_str = '2000-01-01 00:00:00.000000'
date_time_obj = datetime.datetime.strptime(date_time_str, '%Y-%m-%d %H:%M:%S.%f')


#Change Date value to 2000-01-01 if it is str or float or int or bool or complex or bytes datatype
df['Date'] = [date_time_obj if (type(x) == str or type(x) == float or type(x) == int or type(x) == bool or type(x) == complex or type(x) == bytes) else x for x in df['Date']]


#Change Invested value to 0 if it is str or bool or complex or bytes datatype
df['Invested in market from Salary'] = [0 if (type(x) == str or type(x) == bool or type(x) == complex or type(x) == bytes) else x for x in df['Invested in market from Salary']]


#Change Date to Qtr
df['Date'] = df['Date'].dt.to_period("Q")


#get all unique Qtr values
a = df.Date.unique()

#initialise list b and z
b = [] #will have the year-quarter like 2017Q3
z = [] #will have the amount of money invested in that quarter of that year


for i in range (len(a)):
    
    #increase the length of the list b and z by 1
    b = b + ['']
    z = z + ['']
    
    #concatenate the unique date with the word sum. This becomes my new variable
    b[i] = str(a[i])+"sum"
    
    #create a dynamic variable name same as Qtr of the year and store the sum in it
    name = b[i]
    
    #Sum all values of that Qtr and store it in a variable
    value = df.loc[df['Date'] == a[i], 'Invested in market from Salary'].sum()
    dict = {name: value}
    
    #print(b[i],"has", dict[b[i]])
    z[i] = dict[b[i]]
    
#create a dataframe with the new data
dfn = pd.DataFrame({"Date": a, "Quarter":b, "Total Amount Invested":z})
    
#Assign Interest Rate based on year and create a new column in the dataframe dfn
def ifef1(i):
    col = str(i)
    if col[0:4] == '2015':
        return float(8.2)
    if col[0:4] == '2016':
        return float(8.2)
    if col[0:4] =='2017':
        return float(7.75)
    if col[0:4] =='2018':
        return float(7.5)
    if col[0:4] =='2019':
        return float(7.5)
    if col[0:4] =='2020':
        return float(6.5)
    if col[0:4] =='2021':
        return float(6.0)
    else:
        return float(0.0)
    
dfn['Interest Rate'] = dfn['Quarter'].apply(ifef1)


#Calculate number of quarters until now since investment and create a new column Number-of-Qtrs in the dataframe dfn
def function1(i):
    col = str(i)
    now = datetime.datetime.now()
    qtrvar = math.ceil(now.month/3.)
    yearvar = math.ceil(now.year)
    qtrdiff = (yearvar - a.year)*4 + abs((qtrvar - a.quarter))
    test = 5
    return qtrdiff


#Creating new column Number-of-Qtrs through the function1
dfn['Number-of-Qtrs'] = dfn['Date'].apply(function1)

#Creating new column Amount = (1+ ((int rate)/4)/100) to the power Number-of-Qtrs * Principal amount
dfn['Amount'] = ((1+dfn['Interest Rate']/400)**dfn['Number-of-Qtrs'])*dfn['Total Amount Invested']

#Changing amount to int for readability
dfn['Amount'] = dfn['Amount'].astype(int) 

#Summing up all the Amounts to get the overall number
currentbalance = dfn['Amount'].sum()

#Sort the year-quarter values in descending order for readability purpose
dfn = dfn.sort_values(by=['Quarter'])




## Export dataframe to excel without deleting current sheets of excel
import pandas as pd
import numpy as np
from openpyxl import load_workbook

path = 'SIP.xlsx'

book = load_workbook(path)
writer = pd.ExcelWriter(path, engine = 'openpyxl') #version 3.0.4
writer.book = book

df3 = pd.DataFrame(dfn)


df3.to_excel(writer, sheet_name = 'Python')


#The following 2 commands are commented, uncomment it when required to create the sheets on excel 
#writer.save()
#writer.close()


print (dfn)



int64
currentbalance 7406155
      Date    Quarter  Total Amount Invested  Interest Rate  Number-of-Qtrs  \
0   2015Q1  2015Q1sum                   0.00           8.20              23   
1   2015Q2  2015Q2sum                   0.00           8.20              22   
2   2015Q3  2015Q3sum                   0.00           8.20              21   
3   2015Q4  2015Q4sum                   0.00           8.20              20   
4   2016Q1  2016Q1sum               26000.00           8.20              19   
5   2016Q2  2016Q2sum              168000.00           8.20              18   
6   2016Q3  2016Q3sum              535000.00           8.20              17   
7   2016Q4  2016Q4sum              742000.00           8.20              16   
8   2017Q1  2017Q1sum             1060000.00           7.75              15   
9   2017Q2  2017Q2sum              735000.00           7.75              14   
10  2017Q3  2017Q3sum              457400.00           7.75              13   
11  2017Q4  2017Q4sum  

In [170]:
## Export dataframe to excel without deleting current sheets of excel

import pandas as pd
import numpy as np
from openpyxl import load_workbook

path = 'SIP.xlsx'

book = load_workbook(path)
writer = pd.ExcelWriter(path, engine = 'openpyxl') #version 3.0.4
writer.book = book


df3 = pd.DataFrame(dfn)


df3.to_excel(writer, sheet_name = 'Python')

writer.save()
writer.close()